In [1]:
import tomo_challenge as tc
import pandas as pd
import numpy as np
import csv
import os
import json

Found classifier funbins
Found classifier IBandOnly
Found classifier RandomForest
Found classifier Random


In [2]:
import tomo_challenge as tc
#format input data from tomo challenge
bands = 'ugrizy'
is_buzzard = True
if is_buzzard:
    training_file = 'training_buzzard.hdf5'
    testing_file = 'validation_buzzard.hdf5'
    train_frac = 0.13
    test_frac = 0.37
    keep_frac = 0.340
    sample_name = 'buzzard'
if not is_buzzard:
    training_file = 'training.hdf5'
    testing_file = 'validation.hdf5'
    train_frac = 0.084
    test_frac = 0.23
    keep_frac = 0.213
    sample_name = 'DC2'

print(training_file)

include_colors = True
include_errors = True


#application data
data = tc.load_data(testing_file, bands, errors=include_errors, colors=include_colors, size=True, array=False)
data_z = tc.load_redshift(testing_file)
data = pd.DataFrame.from_dict(data)

#adding redshifts
#training_data['specz'] = training_z
data['specz'] = data_z




with open(f'object_ratios_testing_{sample_name}') as json_file:
            ratios = json.load(json_file)
        
ratios_list = []
for i in range(len(ratios)):
    ratios_list.append(ratios[f'{i}'])
data['ratios'] = ratios_list

training_buzzard.hdf5


/global/u2/i/irenem/tomo_challenge/tomo_challenge/data.py:85: UserWarning: Setting inf (undetected) bands to mag=30
  warnings.warn("Setting inf (undetected) bands to mag=30")


In [3]:
#file function
def generate_input_files(directory, filename = 'tpzrun', inputdf = data, tpz_nnr_nnc_split = np.array([0.333, 0.666]), nn_train_val_split = 0.85, overall_split = 0.8, no_nnr = True, sample_frac = keep_frac, subrun = False):

    # directory: The location of the run where you want to save all the files
    # filesname: The name of all the files for this run
    # inputfp: The file path of the input catalog
    # tpz_nnr_nnc_split: The split of the data for TPZ/NNR/NNC training; default is in thirds (or in halves if there is no NNR)
    # nn_train_val_split: Fraction of objects assigned to each NN for training that will be used for training (as opposed to validation)
    # overall_split: The fraction of the input catalog that will be used for training TPZ/NNR/NNC.  The remaining objects will be the "application set"
    # train_frac_lim: Fraction of training sample you would like to use.  Should typically behttps://docs.google.com/presentation/d/1bZtAba8dzczmLLvutQDPrBBSjbTU1KS5hxAXAlq2A-I/edit?usp=sharing 1. 

    # NOTE TO IRENE
    # Don't forget, you will need to replace default_test_train_split with your own function for splitting the indices into training and application indices.  It should split them using overall_split
    # Write a function called get_features that takes a list of indices as an argument and outputs (specz, photometric_features, photometric_feature_errs) for those objects.  photometric_features (and errs) should have shape NxM where N is the number of objects and M is the number of features
    # Write a function to replace hsc_select that returns all the indices of objects that are acceptable for training/fitting.  No NaNs, negative magnitudes, etc.  You may want to apply a signal to noise cut, etc...

    # Here is the run order:
    # 1) Generate Files (will make indexing files and TPZ train/test/application files)
    # 2) Train TPZ (mpiexec -n <NUMCORES> runmlz <filename>.inputs)
    # 3) Run TPZ on application set (mpiexec -n <NUMCORES> runMLZ --modify testfile=<filename>.tpz_app --no_train <filename>.inputs)
    # 4) Generate Files (will make NNR and NNC train/test/application files)
    # 5) Train NNC (nnc = nnclass.nn_classifier(run_folder = <directory>, data_name = <filename>))
    # 6) Run NNC on application set (nnc.fit_app())
    # At this point, your important outputs will be in the .nn_app file (for the TPZ features) and the results_application.dat file within the NNC's output folder (which has all the NNC confidence values)


    #If there isn't a NNR running, just set the default split to 50/50

    if no_nnr and all(tpz_nnr_nnc_split == np.array([0.333, 0.666])):
        tpz_nnr_nnc_split = np.array([0.5, 0.5])

    # =======================
    # GENERATE INDEXING FILES
    # =======================

    if not os.path.isfile(directory + filename + '.train_inds') and not os.path.isfile(directory + filename + '.test_inds'):

        print('Generating fresh indexing files in %s' % directory)

        # indices should contain the integer indices of objects in the catalog that are good for training/testing/etc.
        #If using every object, set cuts=False
        all_indices = data_select_testing(inputdf, unrep=False, sample=sample_name)

        if sample_frac !=1:
            np.random.shuffle(all_indices)
            selected_indices = np.sort(all_indices[:int(len(all_indices)*sample_frac)])
         
        print(len(selected_indices))
        selected_data = inputdf.iloc[selected_indices,:]
       
        training_indices = data_select_testing(selected_data, unrep=True, sample=sample_name)
        testing_indices = data_select_training(selected_data, unrep=True, sample=sample_name)

        # Here, it has split indices into two lists, which are the OVERALL train and test indices (for the whole pipeline, not a single element; this "test_inds" is essentially the application set).
        #train_inds, test_inds = default_test_train_split(indices, splitfrac = overall_split)
        train_inds = training_indices
        test_inds = testing_indices

        print('training and testing index lengths')
        print(len(train_inds), len(test_inds))

        #Shuffle the training indices and assign objects as training objects for TPZ, NNR, and NNC

        scrambled_train = np.arange(len(train_inds))
        np.random.shuffle(scrambled_train)

        tpz_indicator = np.zeros(len(train_inds), dtype = int)
        nnr_indicator = np.zeros(len(train_inds), dtype = int)
        nnc_indicator = np.zeros(len(train_inds), dtype = int)

        tpz_split, nnr_split, nnc_split = np.split(scrambled_train, (tpz_nnr_nnc_split * len(scrambled_train)).astype(int))

        tpz_indicator[tpz_split] = 1
        nnr_indicator[nnr_split] = 1
        nnc_indicator[nnc_split] = 1

       ## Save some files that have flags for what trains on what

        np.savetxt(directory + filename + '.train_inds', np.vstack((train_inds, tpz_indicator, nnr_indicator, nnc_indicator)).T, header = 'train_inds  tpz_train  nnr_train  nnc_train', fmt = '%i  %i  %i  %i')
        np.savetxt(directory + filename + '.app_inds', test_inds, header = 'test_inds', fmt = '%i')

        print('finished: generate indexing files')

        generate_input_files(directory, filename, inputdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, sample_frac, subrun = True)
   # ==================
   # GENERATE TPZ FILES
   # ==================
    elif not os.path.isfile(directory + filename + '.tpz_train') and not os.path.isfile(directory + filename + '.tpz_test'):

        print('Generating fresh TPZ input files in %s' % directory)

        train_inds, tpz_indicator, nnr_indicator, nnc_indicator = np.loadtxt(directory + filename + '.train_inds', unpack = True, dtype = int)

        tpz_indicator = tpz_indicator.astype(bool)
        nnr_indicator = nnr_indicator.astype(bool)
        nnc_indicator = nnc_indicator.astype(bool)

        tpz_train_inds = train_inds[tpz_indicator]
        tpz_test_inds = train_inds[nnr_indicator | nnc_indicator]  # The TPZ test indices are all the objects in the pipeline training set that won't be used by the NNC or NNR

        # Make a function called 'get_features' that returns the object specz's, all relevant photometric features, and their errors
        train_specz, train_features, train_feature_errs = get_features(tpz_train_inds, inputdf, size=False)
        test_specz, test_features, test_feature_errs = get_features(tpz_test_inds, inputdf, size=False)

        header = 'specz g r i z y gr ri iz zy gri riz izy eg er ei ez ey egr eri eiz ezy egri eriz eizy'
        fmt = ' %.10f' + ' %.5f'*5 + ' %.5e' * 7 + ' %.5e'*12

        # Save the TPZ training and test set files - after this step, you're going to have to run TPZ before anything else will happen

        np.savetxt(directory + filename + '.tpz_train', np.hstack((train_specz.reshape(-1,1), train_features, train_feature_errs)), header = header, fmt = fmt)
        np.savetxt(directory + filename + '.tpz_test', np.hstack((test_specz.reshape(-1,1), test_features, test_feature_errs)), header = header, fmt = fmt)
        print('finished: generate tpz files')

        generate_input_files(directory, filename, inputdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, sample_frac, subrun = True)
        #generate_input_files(directory, filename, inputdf, testingdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, train_frac_lim, test_frac_lim, subrun = True)

    # ==================
    # GENERATE NNR FILES
    # ==================

    elif (os.path.isfile(directory + 'output/results/' + filename + '.0.mlz') and not 
            (os.path.isfile(directory + filename + '.nnr_train') or 
                os.path.isfile(directory + filename + '.nnr_test') or 
                os.path.isfile(directory + filename + '.nnr_validate'))):

        # This section likely isn't needed since we aren't running the NNR, but I don't want to accidentally break the rest and it just generates a few extra files

        print('Generating fresh NNR input files in %s' % directory)

        train_inds, tpz_indicator, nnr_indicator, nnc_indicator = np.loadtxt(directory + filename + '.train_inds', dtype = int, unpack = True)

        tpz_indicator = tpz_indicator.astype(bool)
        nnr_indicator = nnr_indicator.astype(bool)
        nnc_indicator = nnc_indicator.astype(bool)

        #Scramble the training objects for the NNR to produce its training and validation files

        train_inds_scrambled = np.copy(train_inds[nnr_indicator])
        np.random.shuffle(train_inds_scrambled)

        nnr_train_inds, nnr_validate_inds = np.split(train_inds_scrambled, [int(nn_train_val_split*len(train_inds_scrambled))])
        nnr_train_inds.sort()
        nnr_validate_inds.sort()

        np.savetxt(directory + filename + '.nnr_train_inds', nnr_train_inds, fmt = '%i')
        np.savetxt(directory + filename + '.nnr_val_inds', nnr_validate_inds, fmt = '%i')

        nnr_test_inds = train_inds[nnc_indicator]

        # Get the features for the correct indices

        nnr_train_specz, nnr_train_features, nnr_train_feature_errs = get_features(nnr_train_inds, inputdf, size=True)
        nnr_validate_specz, nnr_validate_features, nnr_validate_feature_errs = get_features(nnr_validate_inds, inputdf, size=True)
        nnr_test_specz, nnr_test_features, nnr_test_feature_errs = get_features(nnr_test_inds, inputdf, size=True)

        # Load the TPZ outputs because the NNR needs them for training

        tpz_features = np.loadtxt(directory + 'output/results/' + filename + '.0.mlz', usecols = [2,4,6]) # 1,3,5 corresponds to zmode; 2,4,6 corresponds to zmean; is z, zconf, zerr

        nnr_indicator_trans = nnr_indicator[~tpz_indicator]
        nnc_indicator_trans = nnc_indicator[~tpz_indicator]

        # Build the features

        nnr_train_tpz_features, nnr_validate_tpz_features = np.split(tpz_features[nnr_indicator_trans], [int(nn_train_val_split*np.sum(nnr_indicator_trans))])
        nnr_test_tpz_features = tpz_features[nnc_indicator_trans]

        header = 'specz size g r i z y gr ri iz zy gri riz izy zphot zconf zerr eg er ei ez ey egr eri eiz ezy egri eriz eizy'
        fmt = ' %.10f' + ' %.5f'*6 + ' %.5e' * 7 + ' %.5e'*12 + ' %.5e'*3

        # Save all the NNR files

        np.savetxt(directory + filename + '.nnr_train', np.hstack((nnr_train_specz.reshape(-1,1), nnr_train_features, nnr_train_tpz_features, nnr_train_feature_errs)), comments = '', header = header, fmt = fmt)
        np.savetxt(directory + filename + '.nnr_validate', np.hstack((nnr_validate_specz.reshape(-1,1), nnr_validate_features, nnr_validate_tpz_features, nnr_validate_feature_errs)), comments = '', header = header, fmt = fmt)
        np.savetxt(directory + filename + '.nnr_test', np.hstack((nnr_test_specz.reshape(-1,1), nnr_test_features, nnr_test_tpz_features, nnr_test_feature_errs)), comments = '', header = header, fmt = fmt)
        print('finished: generate NNR files')
    
        generate_input_files(directory, filename, inputdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, sample_frac, subrun = True)
        #generate_input_files(directory, filename, inputdf, testingdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, train_frac_lim, test_frac_lim, subrun = True)

    # ==================
    # GENERATE NNC FILES
    # ==================

    elif (os.path.isfile(directory + 'output/results/' + filename + '.0.mlz') and not 
            (os.path.isfile(directory + filename + '.nnc_train') or 
                os.path.isfile(directory + filename + '.nnc_test') or 
                os.path.isfile(directory + filename + '.nnc_validate'))):

        print('Generating fresh NNC input files in %s' % directory)
        
        train_inds, tpz_indicator, nnr_indicator, nnc_indicator = np.loadtxt(directory + filename + '.train_inds', dtype = int, unpack = True)

        tpz_indicator = tpz_indicator.astype(bool)
        nnr_indicator = nnr_indicator.astype(bool)
        nnc_indicator = nnc_indicator.astype(bool)

        # Read in the TPZ features from the NNR test set file

        nnr_test_inds = train_inds[nnc_indicator]
        nnr_test_specz, nnr_test_features, nnr_test_feature_errs = get_features(nnr_test_inds, inputdf, size=True)
        nnr_test_tpz_features = pd.read_csv(directory + filename  + '.nnr_test', delimiter = '\s+', comment = '#')[['zphot', 'zconf', 'zerr']].to_numpy()

        translation_inds = np.arange(len(nnr_test_inds))
        np.random.shuffle(translation_inds)

        # Split the training set in to training and validation

        nnc_train_inds, nnc_validate_inds = np.split(nnr_test_inds[translation_inds], {int(nn_train_val_split*len(nnr_test_inds))})
        train_trans, val_trans = np.split(translation_inds, [int(nn_train_val_split*len(nnr_test_inds))])

        np.savetxt(directory + filename + '.nnc_train_inds', nnc_train_inds, fmt = '%i')
        np.savetxt(directory + filename + '.nnc_val_inds', nnc_validate_inds, fmt = '%i')

        np.savetxt(directory + filename + '.nnc_train_inds', np.vstack((nnc_train_inds, train_trans)).T, fmt = '%i  %i', header = 'HSC_ind  NNR_ind')
        np.savetxt(directory + filename + '.nnc_validate_inds', np.vstack((nnc_validate_inds, val_trans)).T, fmt = '%i  %i', header = 'HSC_ind  NNR_ind')

        # Build the features that will go in the train and validate files

        nnc_train_features = nnr_test_features[train_trans]
        nnc_validate_features = nnr_test_features[val_trans]
        
        nnc_train_feature_errs = nnr_test_feature_errs[train_trans]
        nnc_validate_feature_errs = nnr_test_feature_errs[val_trans]
        
        nnc_train_specz = nnr_test_specz[train_trans]
        nnc_validate_specz = nnr_test_specz[val_trans]
        
        nnc_train_tpz_features = nnr_test_tpz_features[train_trans]
        nnc_validate_tpz_features = nnr_test_tpz_features[val_trans]
        
        header = 'specz size g r i z y gr ri iz zy gri riz izy zphot zconf zerr eg er ei ez ey egr eri eiz ezy egri eriz eizy'
        fmt = ' %.10f' + ' %.5f'*6 + ' %.5e' * 7 + ' %.5e'*12 + ' %.5e'*3 

        # Save the NNC training and validation files

        np.savetxt(directory + filename + '.nnc_train', np.hstack((nnc_train_specz.reshape(-1,1), nnc_train_features, nnc_train_tpz_features, nnc_train_feature_errs)), comments = '', header = header, fmt = fmt)
        np.savetxt(directory + filename + '.nnc_validate', np.hstack((nnc_validate_specz.reshape(-1,1), nnc_validate_features, nnc_validate_tpz_features, nnc_validate_feature_errs)), comments = '', header = header, fmt = fmt)
        # np.savetxt(directory + filename + '.nnc_test', np.hstack((nnc_test_specz.reshape(-1,1), nnc_test_features, nnc_test_tpz_features, nnc_test_feature_errs, nnc_test_features_lo, nnc_test_features_hi, nnc_test_features_frac)), comments = '', header = header, fmt = fmt)

        print('finished: generate NNC files')
        generate_input_files(directory, filename, inputdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, sample_frac, subrun = True)
        #generate_input_files(directory, filename, inputdf, testingdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, train_frac_lim, test_frac_lim, subrun = True)

    # ==================================
    # GENERATE TPZ APPLICATION SET FILES
    # ==================================

    elif not os.path.isfile(directory + filename + '.tpz_app'):

        print('Generating TPZ application set files...')

        app_inds = np.loadtxt(directory + filename + '.app_inds', dtype = int)
        app_specz, app_features, app_feature_errs = get_features(app_inds, inputdf, size=False)

        header = 'specz g r i z y gr ri iz zy gri riz izy eg er ei ez ey egr eri eiz ezy egri eriz eizy'
        fmt = ' %.10f' + ' %.5f'*5 + ' %.5e' * 7 + ' %.5e'*12

        # Generate the TPZ application set

        np.savetxt(directory + filename + '.tpz_app', np.hstack((app_specz.reshape(-1,1), app_features, app_feature_errs)), header = header, fmt = fmt)
        print('finished: generate TPZ application set files')
        generate_input_files(directory, filename, inputdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, sample_frac, subrun = True)
        #generate_input_files(directory, filename, inputdf, testingdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, train_frac_lim, test_frac_lim, subrun = True)

    # ==========================================
   # GENERATE NNC AND NNR APPLICATION SET FILES
   # ==========================================

    elif not os.path.isfile(directory + filename + '.nn_app') and os.path.isfile(directory + 'output/results/' + filename + '.1.mlz'):

        print('Generating NN application set files...')

        app_inds = np.loadtxt(directory + filename + '.app_inds', dtype = int)
        app_specz, app_features, app_feature_errs = get_features(app_inds, inputdf, size=True)

        app_tpz_features = np.loadtxt(directory + 'output/results/' + filename + '.1.mlz', usecols = [2,4,6]) # 1,3,5 corresponds to zmode; 2,4,6 corresponds to zmean; is z, zconf, zerr
       
        header = 'specz size g r i z y gr ri iz zy gri riz izy zphot zconf zerr eg er ei ez ey egr eri eiz ezy egri eriz eizy '
        fmt = ' %.10f' + ' %.5f'*6 + ' %.5e' * 7 + ' %.5e'*12 + ' %.5e'*3

        # Generate the NN application set

        np.savetxt(directory + filename + '.nn_app', np.hstack((app_specz.reshape(-1,1), app_features, app_tpz_features, app_feature_errs)), comments = '', header = header, fmt = fmt)

        print('finished: generate NNC and NNR application set files')
        generate_input_files(directory, filename, inputdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, sample_frac, subrun = True)
        #generate_input_files(directory, filename, inputdf, testingdf, tpz_nnr_nnc_split, nn_train_val_split, overall_split, no_nnr, train_frac_lim, test_frac_lim, subrun = True)

    elif not subrun:
        print('All of these files already exist.  Please delete the input files before continuing.')





def data_select_training(dataframe, unrep, sample):
    '''
    datafram is the pandas dataframe of all the galaxy photometry and true redshift information
    unrep: True or False, do we want to select an unrepresentative training and application sample
    sample: string, either 'DC2' or 'buzzard'
    '''
    
    #dataframe = pd.DataFrame.from_dict(dataframe)

    if unrep == True:
        
        #identify unique ratio values
        unique_ratios = dataframe['ratios'].unique()
        
        #Select the indices of the dataframe to keep
        keep_inds = []
        for i in range(len(unique_ratios)):
            temp_data = dataframe[dataframe['ratios'] == unique_ratios[i]]
            number_to_keep = int(len(temp_data)*(1-unique_ratios[i]))
            indices_to_list = temp_data.index.values.tolist()
            for j in range(0, number_to_keep):
                keep_inds.append(indices_to_list[j])
        
        
        good_data = dataframe.loc[keep_inds,:]
        index_list = good_data.index.values.tolist()
    else:
        index_list = dataframe.index.values.tolist()
    
    return index_list

def data_select_testing(dataframe, unrep, sample):
    '''
    datafram is the pandas dataframe of all the galaxy photometry and true redshift information
    unrep: True or False, do we want to select an unrepresentative training and application sample
    sample: string, either 'DC2' or 'buzzard'
    '''
    #dataframe = pd.DataFrame.from_dict(dataframe)

    if unrep == True:
        
        #identify unique ratio values
        unique_ratios = dataframe['ratios'].unique()
        
        #Select the indices of the dataframe to keep
        keep_inds = []
        for i in range(len(unique_ratios)):
            temp_data = dataframe[dataframe['ratios'] == unique_ratios[i]]
            number_to_keep = int(len(temp_data)*unique_ratios[i])
            indices_to_list = temp_data.index.values.tolist()
            for j in range(0, number_to_keep):
                keep_inds.append(indices_to_list[j])
        
        
        good_data = dataframe.loc[keep_inds,:]
        index_list = good_data.index.values.tolist()
    else:
        index_list = dataframe.index.values.tolist()
    
    return index_list
    

def default_test_train_split(indices, splitfrac):

    split_len = len(indices)*splitfrac
    split_len = int(split_len)

    train_inds = indices[0:split_len]
    test_inds = indices[split_len:]

    return train_inds, test_inds

def get_features(indices, dataframe, size):

    #dataframe = pd.DataFrame.from_dict(dataframe)

    speczs = dataframe['specz'].loc[indices]
    speczs = speczs.to_numpy()

    dataframe['gri'] = dataframe['gr']-dataframe['ri']
    dataframe['egri'] = np.sqrt((dataframe['g_err']**2)+4*(dataframe['r_err']**2)+(dataframe['i_err']**2))
    dataframe['riz'] = dataframe['ri']-dataframe['iz']
    dataframe['eriz'] = np.sqrt((dataframe['r_err']**2)+4*(dataframe['i_err']**2)+(dataframe['z_err']**2))
    dataframe['izy'] = dataframe['iz']-dataframe['zy']
    dataframe['eizy'] = np.sqrt((dataframe['i_err']**2)+4*(dataframe['z_err']**2)+(dataframe['y_err']**2))

    if size:
        features = dataframe[['mcal_T', 'g', 'r', 'i', 'z', 'y', 'gr', 'ri', 'iz', 'zy', 'gri', 'riz', 'izy']].loc[indices]
    if not size:
        features = dataframe[['g', 'r', 'i', 'z', 'y', 'gr', 'ri', 'iz', 'zy', 'gri', 'riz', 'izy']].loc[indices]
    features = features.to_numpy()
    feature_errs = dataframe[['g_err', 'r_err', 'i_err', 'z_err', 'y_err', 'gr_err', 'ri_err', 'iz_err', 'zy_err', 'egri', 'eriz', 'eizy']].loc[indices]
    feature_errs = feature_errs.to_numpy()

    return speczs, features, feature_errs

In [4]:
generate_input_files(directory=f'nn_class/unrep/Buzzard/', filename = 'tpzrun', inputdf = data, tpz_nnr_nnc_split = np.array([0.333, 0.666]), nn_train_val_split = 0.85, overall_split = 0.8, no_nnr = True, sample_frac = keep_frac, subrun = False)

Generating fresh NNR input files in nn_class/unrep/Buzzard/
finished: generate NNR files
Generating fresh NNC input files in nn_class/unrep/Buzzard/
finished: generate NNC files
Generating NN application set files...
finished: generate NNC and NNR application set files
